# Fitting Hyperbolas based in locations found with YOLO Model

In [1]:
#import libraries
import numpy as np
from ultralytics import YOLO
import os

#Setting Working directory
import sys
from pathlib import Path

sys.path.append(str(Path.cwd().parent))

#Import the config file so that only the Filename needs to be changed in the _read_segy function
from config import *

In [2]:
#Load trained Model
model = YOLO(YOLO_MODEL_DIR)

In [3]:
from Pipeline.Datatoolkit import DatatoolKit

dk = DatatoolKit(TEST_FILE_DIR , "UG3DQUERUNTERZUG.SGY")
file = dk.LoadSGY()
df = dk.create_df(file)

In [4]:
def extract_bboxes(prediction_folder)-> list:
    results = model.predict(source=prediction_folder, save = True, conf=0.1)
    all_boxes = {}

    for result in results:
        img_name = os.path.basename(result.path)
        if result.boxes is not None:
            boxes = result.boxes.xyxy.cpu().numpy()
            all_boxes[img_name] = boxes# Bounding boxes in (x1, y1, x2, y2) format
        else:
            all_boxes[img_name] = np.empty((0,4))       
        
    return all_boxes

In [5]:
boxes = extract_bboxes(TEST_PIC_DIR)


image 1/2 c:\vscode\PAINDGPR\Data\Testdata\pictures\UG3DQUERUNTERZUG.SGY_inline_18.png: 384x640 5 hyperbolas, 72.5ms
image 2/2 c:\vscode\PAINDGPR\Data\Testdata\pictures\frame_0.jpg: 448x640 5 hyperbolas, 84.6ms
Speed: 2.5ms preprocess, 78.6ms inference, 1.3ms postprocess per image at shape (1, 3, 448, 640)
Results saved to C:\vscode\PAINDGPR\Notebooks\runs\detect\predict6


In [6]:
print(boxes)

{'UG3DQUERUNTERZUG.SGY_inline_18.png': array([[      675.2,      4.3947,         711,      52.558],
       [     560.25,      8.0632,      609.38,      53.285],
       [     506.06,      5.9797,      553.49,      53.632],
       [     347.53,      74.568,      493.78,      130.36],
       [     609.33,      7.9033,      663.83,      51.816]], dtype=float32), 'frame_0.jpg': array([[     151.06,      48.751,      194.37,      103.59],
       [     219.82,      44.851,      263.55,      108.34],
       [     653.49,      47.296,      693.86,      112.03],
       [     721.31,      47.675,       756.3,       104.8],
       [      583.1,      46.415,      620.13,      95.713]], dtype=float32)}


In [12]:
#translate bboxes coordinates to sgy date

def bbox_to_data(df, bbox:dict,sample_interval=None):

    results = {}

    for img_name, box_list in bbox.items():
        if "_inline_" not in img_name:
            continue
        inline_nr = int(img_name.split("_inline_")[1].split(".")[0])
        results[img_name] = []

        for bbox in box_list:
            x1,y1,x2,y2 = map(int, bbox)

            sub = df[(df["inline"] == inline_nr) & (df["crossline"].between(x1,x2))]
            sub = sub.sort_values("crossline")
            if sub.empty:
                continue

            amp_slice = np.vstack([
                np.array(row["Amplitude"])[y1:y2] for _, row in sub.iterrows()
            ]).T

            
            if sample_interval is not None:
                time_start_us = y1 * sample_interval
                time_end_us = y2 * sample_interval
            else:
                time_start_us = time_end_us = None

            results[img_name].append({
                "bbox": bbox,
                "inline": inline_nr,
                "crossline_min": int(sub["crossline"].min()),
                "crossline_max": int(sub["crossline"].max()),
                "sample_start": y1,
                "sample_end": y2,
                "time_start_us": time_start_us,
                "time_end_us": time_end_us,
                "amplitudes": amp_slice
            })

    return results


In [13]:
results = bbox_to_data(df, boxes)

In [14]:
from pprint import pprint
pprint(results)

{'UG3DQUERUNTERZUG.SGY_inline_18.png': [{'amplitudes': array([[-10005, -10476, -10493, ..., -10000, -10117, -10280],
       [ -5132,  -6361,  -6507, ...,  -4448,  -3707,  -3276],
       [ -1477,  -2626,  -2711, ...,   -302,    525,   1211],
       ...,
       [  3619,   1795,    208, ...,   5750,   6941,   6843],
       [  3774,   2249,    451, ...,   2483,   5099,   6165],
       [  2285,   2020,   1203, ...,   -880,   1962,   3753]], shape=(48, 37), dtype=int16),
                                         'bbox': array([      675.2,      4.3947,         711,      52.558], dtype=float32),
                                         'crossline_max': 711,
                                         'crossline_min': 675,
                                         'inline': 18,
                                         'sample_end': 52,
                                         'sample_start': 4,
                                         'time_end_us': None,
                                         't

## Edge Detection on bounding boxes

https://www.ultralytics.com/blog/edge-detection-in-image-processing-explained

https://blog.roboflow.com/edge-detection/



#### Canny Edge Detection

In [19]:
import cv2
import numpy as np

sigma = 1
low = 0.3
high = 0.7

edge_results = {}

for img_name , entries in results.items():
    edge_results[img_name] = []

    for e in entries:
        amp = e["amplitudes"].astype(float)

        #normalize
        img = (amp - np.min(amp)) / (np.max(amp) - np.min(amp) + 1e-9)
        img =  (img * 255).astype(np.uint8)

        blurred = cv2.GaussianBlur(img, (3,3), sigma)
        edges = cv2.Canny(blurred, int(low *255), int(high*255))

        e_new = e.copy()
        e_new["edges"] = edges
        edge_results[img_name].append(e_new)


In [20]:
import matplotlib.pyplot as plt

outputdir = EDGE_DETECT_DIR



for img_name, entries in edge_results.items():
    base = os.path.splitext(img_name)[0]
    for idx, e in enumerate(entries):
        if "edges" not in e:
            continue

        edges = e["edges"]
        if edges.dtype != np.uint8:
            edges = np.clip(edges, 0, 255).astype(np.uint8)

        out_path = os.path.join(outputdir, f"{base}_bbox{idx+1}_edges.png")
        cv2.imwrite(out_path, edges)
        